In [28]:
# Compare how many more disk giant stars are there in APOGEE DR17
import astropy
from astropy.table import Table
import numpy as np
import pandas as pd
import h5py

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import ascii, fits

import matplotlib
import matplotlib.pyplot as plt
label_size = 24
matplotlib.rc('font', size=label_size) 

In [15]:
# import full APOGEE DR17 data
tab = Table.read('../../Fa2020_MW_substructure/data/allStarLite-dr17-synspec_rev1.fits',1)
names = [name for name in tab.colnames if len(tab[name].shape) <= 1]
df_a17 = tab[names].to_pandas()

In [3]:
# import astroNN DR17 data
# tab = Table.read('../../Fa2020_MW_substructure/data/apogee_astroNN-DR17.fits')
# names = [name for name in tab.colnames if len(tab[name].shape) <= 1]
# df_aNN17 = tab[names].to_pandas()

In [8]:
# Apply the cuts on the astroNN DR17 data
print(f"Total number of targets in DR17: {len(df_a17)}")

# Get rid of repeated sources
badbits = 2**4
qual_a17 = np.bitwise_and(df_a17['EXTRATARG'],badbits) == 0
print(f"removing repeated sources gives {qual_a17.sum()} stars.")

# Low log g cut
low_grav_a17 = (df_a17['LOGG'] > 0) & (df_a17['LOGG'] < 2.2)
print(f"log g cut based on APOGEE values gives {low_grav_a17.sum()} stars.")

# Must have gaia parallax
gaia_plx_a17 = (np.isfinite(df_a17['GAIAEDR3_PARALLAX']))
print(f"requirement for gaia plx gives {gaia_plx_a17.sum()} stars.")

# Put everything together
allcut_a17 = qual_a17 & low_grav_a17 & gaia_plx_a17
print(f"all cuts together give {allcut_a17.sum()} stars.")

Total number of targets in DR17: 733901
removing repeated sources gives 657135 stars.
log g cut based on APOGEE values gives 159766 stars.
requirement for gaia plx gives 720894 stars.
all cuts together give 146751 stars.


In [63]:
df_a17.loc[allcut_a17,'TELESCOPE'].value_counts()

b'apo25m'    84355
b'lco25m'    62217
b'apo1m'       179
Name: TELESCOPE, dtype: int64

In [17]:
len(tab.colnames)

191

In [61]:
# Export the gaia source ids of these stars for later query for WISE photometry
df_a17.loc[:,['APOGEE_ID','GAIAEDR3_SOURCE_ID']].to_csv('a17_all_aid_g3id.csv',index=False)
# df_a17.loc[allcut_a17,['APOGEE_ID','GAIAEDR3_SOURCE_ID']].to_csv('a17_candidate_aid_g3id.csv',index=False)

In [29]:
hdu = fits.open('../spec_download_norm/data/spec_plx_a17cand_allwise_match.fits')
wise_data = hdu[1].data
hdu.close()

In [40]:
wise_data = Table.read('../spec_download_norm/data/spec_plx_a17cand_allwise_match.fits',1)

In [41]:
np.shape(wise_data)

(146751,)

In [45]:
wise_data

apogee_id,source_id,original_ext_source_id,angular_distance,xm_flag,allwise_oid,number_of_neighbours,number_of_mates,designation,w1mpro,w2mpro,w3mpro,w4mpro,var_flag
,,,arcsec,,,,,,mag,mag,mag,mag,
bytes21,int64,bytes255,float32,int16,int32,int16,int16,bytes19,float64,float64,float64,float64,bytes4
b'2M00000002+7417074',538028216707715712,J000000.04+741707.5,0.0262377,8,733278841,1,0,J000000.04+741707.5,7.257,7.353,7.148,6.992,3110
b'2M00000035-7323394',--,,nan,--,--,--,--,,nan,nan,nan,nan,
b'2M00000317+5821383',422737314373993984,J000003.17+582138.3,0.032791663,8,689699320,1,0,J000003.17+582138.3,6.804,6.951,6.817,6.7,2200
b'2M00000546+6152107',429530195985183744,J000005.49+615210.7,0.10692527,8,701716063,1,0,J000005.49+615210.7,7.523,7.682,7.568,7.633,220n
b'2M00000717-7213573',4701763512829847808,J000007.17-721357.2,0.04410313,8,20002330,1,0,J000007.17-721357.2,14.342,14.443,12.503,8.553,00nn
b'2M00000797+6436119',431771039351005184,J000007.99+643612.0,0.047733244,8,710201285,1,0,J000007.99+643612.0,11.253,11.25,11.245,9.571,99nn
b'2M00001493+5534446',420513826985325696,J000014.93+553444.6,0.048711292,8,679145295,1,0,J000014.93+553444.6,9.168,9.314,9.125,8.655,011n
b'2M00001649-7342309',4689378643918560128,J000016.49-734230.9,0.11712436,8,16683775,1,0,J000016.49-734230.9,14.58,14.648,12.556,9.187,00nn


In [46]:
wise_data.rename_column('apogee_id','APOGEE_ID')

In [49]:
wise_data['APOGEE_ID']

b'2M00000002+7417074'
b'2M00000035-7323394'
b'2M00000317+5821383'
b'2M00000546+6152107'
b'2M00000717-7213573'
b'2M00000797+6436119'
b'2M00001493+5534446'
b'2M00001649-7342309'
b'2M00001653+5540107'
b'2M00001719+6221324'
b'2M00002005+5703467'


In [50]:
df_a17.loc[allcut_a17,'APOGEE_ID']

1         b'2M00000002+7417074'
5         b'2M00000035-7323394'
12        b'2M00000317+5821383'
19        b'2M00000546+6152107'
24        b'2M00000717-7213573'
                  ...          
733887    b'2M23595606+6251121'
733888    b'2M23595637-7223071'
733891    b'2M23595676+7918072'
733895    b'2M23595867-7339039'
733899    b'2M23595949-7342592'
Name: APOGEE_ID, Length: 146751, dtype: object

In [55]:
ii= 0
print(wise_data['APOGEE_ID'][ii], df_a17.loc[allcut_a17,'APOGEE_ID'].values[ii])
print(wise_data['APOGEE_ID'][ii] == df_a17.loc[allcut_a17,'APOGEE_ID'].values[ii])
print(wise_data['source_id'][ii], df_a17.loc[allcut_a17,'GAIAEDR3_SOURCE_ID'].values[ii])
print(wise_data['source_id'][ii] == df_a17.loc[allcut_a17,'GAIAEDR3_SOURCE_ID'].values[ii])

b'2M00000002+7417074' b'2M00000002+7417074'
b'2M00000002+7417074' b'2M00000002+7417074'
False
538028216707715712 538028216707715712
True


In [57]:
print(wise_data['source_id'][1], np.isfinite(wise_data['source_id'][1]))

-- --


In [58]:
wise_data['source_id'].mask

array([False,  True, False, ..., False, False, False])

In [60]:
wise_data[~wise_data['source_id'].mask]

APOGEE_ID,source_id,original_ext_source_id,angular_distance,xm_flag,allwise_oid,number_of_neighbours,number_of_mates,designation,w1mpro,w2mpro,w3mpro,w4mpro,var_flag
,,,arcsec,,,,,,mag,mag,mag,mag,
bytes21,int64,bytes255,float32,int16,int32,int16,int16,bytes19,float64,float64,float64,float64,bytes4
b'2M00000002+7417074',538028216707715712,J000000.04+741707.5,0.0262377,8,733278841,1,0,J000000.04+741707.5,7.257,7.353,7.148,6.992,3110
b'2M00000317+5821383',422737314373993984,J000003.17+582138.3,0.032791663,8,689699320,1,0,J000003.17+582138.3,6.804,6.951,6.817,6.7,2200
b'2M00000546+6152107',429530195985183744,J000005.49+615210.7,0.10692527,8,701716063,1,0,J000005.49+615210.7,7.523,7.682,7.568,7.633,220n
b'2M00000717-7213573',4701763512829847808,J000007.17-721357.2,0.04410313,8,20002330,1,0,J000007.17-721357.2,14.342,14.443,12.503,8.553,00nn
b'2M00000797+6436119',431771039351005184,J000007.99+643612.0,0.047733244,8,710201285,1,0,J000007.99+643612.0,11.253,11.25,11.245,9.571,99nn
b'2M00001493+5534446',420513826985325696,J000014.93+553444.6,0.048711292,8,679145295,1,0,J000014.93+553444.6,9.168,9.314,9.125,8.655,011n
b'2M00001649-7342309',4689378643918560128,J000016.49-734230.9,0.11712436,8,16683775,1,0,J000016.49-734230.9,14.58,14.648,12.556,9.187,00nn
b'2M00001719+6221324',429941795599771136,J000017.19+622132.4,0.011615241,8,703285472,1,0,J000017.19+622132.4,8.957,9.052,8.529,7.194,1100
